In [1]:
import os , sys
# ensure parent folder is in Python path
sys.path.append("/home/ujjwal/learning/fastapi_learn")

# change to parent folder of 'basic'
os.chdir("/home/ujjwal/learning/fastapi_learn/prediction_with_ml_model")

!pwd


/home/ujjwal/learning/fastapi_learn/prediction_with_ml_model


In [3]:
import pandas as pd
url = 'data/bitcoin.csv'

In [4]:
!pwd

/home/ujjwal/learning/fastapi_learn/prediction_with_ml_model


In [85]:
df= pd.read_csv(url)
df

,Start,End,Open,High,Low,Close,Volume,Market Cap
0,2025-08-27,2025-08-28,111661.0000,112600.0000,110445.0000,111194.0000,5.776033e+10,2.219860e+12
1,2025-08-26,2025-08-27,109904.0000,112300.0000,109118.0000,111848.0000,8.225304e+10,2.196098e+12
2,2025-08-25,2025-08-26,113422.0000,113630.0000,109478.0000,110167.0000,7.821957e+10,2.226762e+12
3,2025-08-24,2025-08-25,115378.0000,115667.0000,112155.0000,113400.0000,4.010159e+10,2.280078e+12
4,2025-08-23,2025-08-24,116893.0000,116992.0000,114622.0000,115419.0000,6.411995e+10,2.299783e+12
...,...,...,...,...,...,...,...,...
5504,2010-08-02,2010-08-03,0.0600,0.0600,0.0600,0.0600,0.000000e+00,2.146395e+05
5505,2010-08-01,2010-08-02,0.0611,0.0611,0.0611,0.0611,0.000000e+00,2.175409e+05
5506,2010-07-31,2010-08-01,0.0679,0.0679,0.0679,0.0679,0.000000e+00,2.409992e+05
5507,2010-07-30,2010-07-31,0.0627,0.0627,0.0627,0.0627,0.000000e+00,2.225427e+05


### only doing for fastapi so dont care of accuracy 

#### work_flow

1) check missing values 
2) date is form 2025 to 2010 , so we inverse this to , 20210 to 2025 for ascending order
2) convert date  to numerical value so that ml can understand 
3) sort by date 

4) drop market cap 
5) keep features [ date , open , high , low , volume]
6) targer = close 
7) adding moving average (not much more , dont want to go more in detailed)

11) save with .pkl file 

12) now use this model to preict the future inputs , eg 2026-08-01 then 


### model used 
 Gradient Boosting 

 ### note we dont do not split dataset here , just trained full dataset and predict


In [86]:
#check the missing value
df.isnull().sum()

Start         0
End           0
Open          0
High          0
Low           0
Close         0
Volume        0
Market Cap    0
dtype: int64

In [87]:
print(df.columns)
## ascending order 
sorted_df= df.iloc[::-1].reset_index(drop= True)
sorted_df

Index(['Start', 'End', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')


,Start,End,Open,High,Low,Close,Volume,Market Cap
0,2010-07-29,2010-07-30,0.0699,0.0699,0.0699,0.0699,0.000000e+00,2.466115e+05
1,2010-07-30,2010-07-31,0.0627,0.0627,0.0627,0.0627,0.000000e+00,2.225427e+05
2,2010-07-31,2010-08-01,0.0679,0.0679,0.0679,0.0679,0.000000e+00,2.409992e+05
3,2010-08-01,2010-08-02,0.0611,0.0611,0.0611,0.0611,0.000000e+00,2.175409e+05
4,2010-08-02,2010-08-03,0.0600,0.0600,0.0600,0.0600,0.000000e+00,2.146395e+05
...,...,...,...,...,...,...,...,...
5504,2025-08-23,2025-08-24,116893.0000,116992.0000,114622.0000,115419.0000,6.411995e+10,2.299783e+12
5505,2025-08-24,2025-08-25,115378.0000,115667.0000,112155.0000,113400.0000,4.010159e+10,2.280078e+12
5506,2025-08-25,2025-08-26,113422.0000,113630.0000,109478.0000,110167.0000,7.821957e+10,2.226762e+12
5507,2025-08-26,2025-08-27,109904.0000,112300.0000,109118.0000,111848.0000,8.225304e+10,2.196098e+12


In [88]:
sorted_df['Date']=pd.to_datetime(sorted_df['Start']).map(pd.Timestamp.toordinal)
sorted_df.head(2)

,Start,End,Open,High,Low,Close,Volume,Market Cap,Date
0,2010-07-29,2010-07-30,0.0699,0.0699,0.0699,0.0699,0.0,246611.532344,733982
1,2010-07-30,2010-07-31,0.0627,0.0627,0.0627,0.0627,0.0,222542.677500,733983


In [89]:
## drop , start , end marketcap
sorted_df.drop(columns=['Start','End', 'Market Cap'] , inplace= True)
sorted_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Date'], dtype='object')

In [90]:
# Using lag 

sorted_df['Close_lag1'] = sorted_df['Close'].shift(1)
sorted_df['Close_lag2'] = sorted_df['Close'].shift(2)
sorted_df['Close_lag3'] = sorted_df['Close'].shift(3)

# moving average
sorted_df['MA3'] = sorted_df['Close'].rolling(3).mean()

# Drop rows with NaN
sorted_df = sorted_df.dropna()


In [91]:
## features and target 
features = ['Date', 'Close_lag1', 'Close_lag2', 'Close_lag3', 'MA3']
target = 'Close'

X = sorted_df[features]
y = sorted_df[target]

X.head(2)

,Date,Close_lag1,Close_lag2,Close_lag3,MA3
3,733985,0.0679,0.0627,0.0699,0.0639
4,733986,0.0611,0.0679,0.0627,0.0630


In [94]:
#imports 
from sklearn.ensemble import GradientBoostingRegressor
import pickle 

model = GradientBoostingRegressor()
model.fit(X,y)



sorted_df.to_pickle("processed_df.pkl")

#save model 
with open("bitcoin_model.pkl" , "wb") as f : 
    pickle.dump(model,f)

*************

In [109]:
%%writefile bitcoin.py 
from datetime import timedelta
import pandas as pd
import pickle

# Load preprocessed DataFrame
sorted_df = pd.read_pickle("processed_df.pkl")

# Load trained model
with open("bitcoin_model.pkl", "rb") as f:
    model = pickle.load(f)


def predict_future_close(user_date_str):
    future_df = sorted_df.copy()
    target_date = pd.to_datetime(user_date_str)
    
    last_known_date = pd.Timestamp.fromordinal(int(future_df['Date'].iloc[-1]))
    
    while last_known_date < target_date:
        lag1 = future_df.iloc[-1]['Close']
        lag2 = future_df.iloc[-2]['Close']
        lag3 = future_df.iloc[-3]['Close']
        ma3 = (lag1 + lag2 + lag3) / 3
        
        next_date = last_known_date + timedelta(days=1)
        next_date_ordinal = next_date.toordinal()
        
        # Use DataFrame to match feature names
        X_pred = pd.DataFrame([[next_date_ordinal, lag1, lag2, lag3, ma3]],
                              columns=['Date', 'Close_lag1', 'Close_lag2', 'Close_lag3', 'MA3'])
        
        predicted_close = model.predict(X_pred)[0]
        
        # Append row using concat
        new_row = pd.DataFrame({'Date': [next_date_ordinal], 'Close': [predicted_close]})
        future_df = pd.concat([future_df, new_row], ignore_index=True)
        
        last_known_date = next_date
    
    return predicted_close


Writing bitcoin.py


In [111]:
%run bitcoin.py


In [112]:
predict_future_close("2026-01-05")


np.float64(110836.93971516743)